In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
from operator import itemgetter 
import os

from tqdm import tqdm

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
def load_vdb_and_retriever(path="../vectorstore/historia_ed_financeira",
                           k=4):
    embedding_size = 1536
    embedding_model = "text-embedding-3-small"
    embeddings = OpenAIEmbeddings(model=embedding_model, dimensions=embedding_size)
    
    vdb = FAISS.load_local(path, 
                           embeddings, 
                           allow_dangerous_deserialization=True)
    
    retriever = vdb.as_retriever(search_kwargs={"k": k})
    
    return vdb, retriever
    

### RAG (Retrieval-Augmented Generation)

A técnica utilizada aqui consiste em utilizar o mecanismo de busca baseado nos ``embeddings`` para que se encontre os documentos mais próximos semanticamente para a geração de texto. Dessa forma, o modelo consegue gerar textos que se baseiam na informação contida nos documentos mais próximos.

<img src="../imgs/rag.png" width="600">

In [3]:
vdb, retriever = load_vdb_and_retriever()

retriever.invoke("Qual o peso de um sonho?")

[Document(id='83521480-8227-4ff7-ad64-5e49b407aff3', metadata={'source': '../data/educacao_financeira.pdf', 'page': 48}, page_content='3636\nO que pesa mais, um quilo de algodão ou um quilo de chum-\nbo? Muita gente responde chumbo ao ouvir esta pergunta, \nmas, na verdade, os dois pesam a mesma coisa: um quilo. \nNa hora de pensar nas nossas despesas, também podemos \nficar confusos e acabar dando pesos equivocados a cada coi-\nsa. Às vezes, achamos que estamos gastando muito em uma \ncoisa, quando o que está pesando no nosso bolso na verdade \né outra. \nquaNTO pesa O seu sONhO? \nVocê e sua família devem ter alguns sonhos e projetos que pa-\nrecem muito distantes porque nunca sobra dinheiro suficien-\nte no final do mês para realizá-los. O que fazer nesse caso?\nAntes de tudo, seria bom vocês elegerem um dos projetos que \nquerem realizar para ter um foco bem concreto. Isso ajuda a \nmanter o esforço da família para conseguir o dinheiro neces-\nsário. Escolheu um foco?\nAgora você v

In [6]:
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from typing import List, Dict

def format_docs(docs: List[Document]) -> str:
    return "\n".join([x.page_content for x in docs])

vdb, retriever = load_vdb_and_retriever()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
# llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)


system_prompt = """
Você é um assistente de IA que vai tirar dúvidas sobre educação financeira e história. 

Além disso, aqui está um conteudo extra sobre educação financeira e/ou história:

[Conteudo extra]
{extra_content}
[Final do conteudo extra]

--------------------------------------------
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

chain = (
    {
        "input": itemgetter("input"),
        "extra_content": itemgetter("input") | retriever | RunnableLambda(format_docs)
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

In [5]:
response = chain.invoke({"input": "Me fale sobre o peso de um sonho"})

In [7]:
print(response)

O "peso de um sonho" é uma metáfora que se refere à importância e ao impacto que um sonho ou objetivo pode ter na vida de uma pessoa. Assim como na analogia do quilo de algodão e o quilo de chumbo, onde ambos pesam a mesma coisa, os sonhos podem parecer diferentes em termos de dificuldade ou importância, mas todos têm um peso emocional e prático em nossas vidas.

Quando falamos sobre o peso de um sonho, estamos nos referindo a vários aspectos:

1. **Prioridade**: Alguns sonhos são mais importantes do que outros e podem exigir mais esforço e recursos para serem alcançados. Identificar quais sonhos são prioritários ajuda a direcionar energia e planejamento.

2. **Recursos Necessários**: Realizar um sonho muitas vezes requer tempo, dinheiro e esforço. O "peso" pode se referir à quantidade de recursos que você precisa alocar para alcançar esse sonho.

3. **Impacto Emocional**: Sonhos podem ter um peso emocional significativo. Eles podem trazer felicidade, motivação e um senso de propósito,

In [8]:
async for chunk in chain.astream({"input": "Me fale sobre o peso de um sonho"}):
    print(chunk, end="", flush=True)

O "peso de um sonho" é uma metáfora que se refere à importância e ao impacto que um sonho ou objetivo pode ter na vida de uma pessoa. Assim como na pergunta clássica sobre o peso de um quilo de algodão versus um quilo de chumbo, onde ambos têm o mesmo peso, os sonhos podem parecer diferentes em termos de valor ou dificuldade, mas todos têm um peso emocional e prático em nossas vidas.

Quando falamos sobre o peso de um sonho, podemos considerar alguns aspectos:

1. **Importância Pessoal**: Cada sonho tem um significado único para cada indivíduo. O que pode ser um sonho leve e fácil de alcançar para uma pessoa pode ser um peso significativo para outra, dependendo de suas circunstâncias e prioridades.

2. **Planejamento e Esforço**: Realizar um sonho muitas vezes requer planejamento, esforço e sacrifício. O "peso" aqui se refere ao trabalho que é necessário para transformar um sonho em realidade. Isso pode incluir economizar dinheiro, investir tempo e energia, ou superar obstáculos.

3. *

<h1> Exercícios

- Criar um bot de perguntas e respostas com as playlists do tutorial de embbedings
- usar stremlit (sugestão: usar LCEL)

<h1> Resolução

criar documentos com novas playlists

In [16]:
embedding_size = 1536
embedding_model = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(model=embedding_model, dimensions=embedding_size)

In [17]:
from langchain_community.document_loaders import YoutubeLoader
from pytube import Playlist
from youtube_transcript_api import NoTranscriptFound

In [18]:
playlist_defi = Playlist("https://www.youtube.com/watch?v=PHpryYvBJZU&list=PL_PXtZriHU_jS9WX_zE63WxJR2QGFCSSd")
playlist_mf = Playlist("https://www.youtube.com/watch?v=VQEKO9nokUg&list=PLRYmNZ_ktVo9lTIrJvGgDFNQ0nzIY-6n9")

urls_defi = playlist_defi.video_urls
urls_mf = playlist_mf.video_urls

In [19]:
def cria_documento(urls):
    documents = []
    for video in urls:
        loader = YoutubeLoader.from_youtube_url(video, add_video_info=False, language=["pt"])
        try:
            doc = loader.load()
            if len(doc) > 0:
                documents.append(doc[0])  # Adiciona a transcrição ao documents
        except NoTranscriptFound:
            print(f"Nenhuma transcrição encontrada para o vídeo {video} em pt-BR.")
        except Exception as e:
            print(f"Erro ao carregar o vídeo {video}: {e}")
    return documents

In [20]:
doc_defi = cria_documento(urls_defi)
doc_mf = cria_documento(urls_mf)

In [23]:
all_docs = doc_defi + doc_mf

Criando vector store

In [24]:
vector_db = FAISS.from_documents(all_docs, embeddings)

In [25]:
vector_db.save_local("../rag/defi_e_mf")

In [26]:
vdb = FAISS.load_local("../rag/defi_e_mf", embeddings, allow_dangerous_deserialization=True)

In [29]:
def load_vdb_and_retriever(path="../rag/defi_e_mf",
                           k=4):
    embedding_size = 1536
    embedding_model = "text-embedding-3-small"
    embeddings = OpenAIEmbeddings(model=embedding_model, dimensions=embedding_size)
    
    vdb = FAISS.load_local(path, 
                           embeddings, 
                           allow_dangerous_deserialization=True)
    
    retriever = vdb.as_retriever(search_kwargs={"k": k})
    
    return vdb, retriever

In [30]:
vdb, retriever = load_vdb_and_retriever()

retriever.invoke("o que é mercado financeiro?")

[Document(id='ee2e0574-c830-4f5d-ad12-1b26048bc55a', metadata={'source': 'YjWSGFu6Ezg'}, page_content="tá começando a aula de hoje vamos comentar nesse encontro sobre mercado de capitais vamos explicar que que é isso como que funciona e mercado primário e secundário de ações a gente vai falar bastante do mercado acionário também hoje um sempre nosso material que a se baseando no livro do whatsapp do assaf neto mercado financeiro 14ª edição de 2018 a idade do atlas só para contextualizar aonde que a gente tá né a gente discutiu lá na primeira aula e na segunda aula do curso que o mercado financeiro ele é composto por outros mercados tá ele é composto pelo mercado monetário e a gente já esse do mercado monetário aonde a gente controla ali que desde a economia é que a gente tem um bar sem como é comum um player muito ativo um papel muito pote nesse mercado a gente viu também já estudamos o mercado de crédito certo é onde ocorrem a intermediação financeira de recursos via credores e devedo

In [31]:
from langchain_core.runnables import RunnableLambda
from langchain_core.documents import Document
from typing import List, Dict

def format_docs(docs: List[Document]) -> str:
    return "\n".join([x.page_content for x in docs])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

system_prompt = """
Você é um assistente de IA que vai tirar dúvidas sobre finanças descentralizadas e mercado financeiro. 

Além disso, aqui está um conteudo extra sobre finanças descentralizadas e/ou mercado financeiro:

[Conteudo extra]
{extra_content}
[Final do conteudo extra]

--------------------------------------------
"""

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

chain = (
{
    "input" : itemgetter("input"),
    "extra_content": itemgetter("input") | retriever | RunnableLambda(format_docs)
}
| prompt
| llm
| StrOutputParser()
)


In [33]:
response = chain.invoke({"input": "Me fale um pouco sobre finanças descentralizadas (DEFI)"})

In [34]:
print(response)

Finanças Descentralizadas, ou DeFi (do inglês "Decentralized Finance"), referem-se a um ecossistema de aplicações financeiras que operam em redes blockchain, principalmente na Ethereum. O objetivo do DeFi é criar um sistema financeiro mais acessível, transparente e sem intermediários, permitindo que qualquer pessoa com uma conexão à internet possa participar.

Aqui estão alguns pontos-chave sobre DeFi:

1. **Descentralização**: Ao contrário do sistema financeiro tradicional, que depende de bancos e instituições financeiras centralizadas, o DeFi utiliza contratos inteligentes (smart contracts) para automatizar e executar transações sem a necessidade de intermediários.

2. **Acessibilidade**: Qualquer pessoa pode acessar serviços financeiros através de plataformas DeFi, desde que tenha uma carteira digital e criptomoedas. Isso é especialmente importante para pessoas em regiões onde o acesso a serviços bancários é limitado.

3. **Variedade de Serviços**: O DeFi oferece uma ampla gama de s

In [35]:
async for chunk in chain.astream({"input": "Me fale um pouco sobre finanças descentralizadas (DEFI)"}):
    print(chunk, end="", flush=True)

Finanças descentralizadas, ou DeFi (do inglês "Decentralized Finance"), referem-se a um ecossistema financeiro que opera em plataformas de blockchain, permitindo a realização de transações financeiras sem a necessidade de intermediários tradicionais, como bancos ou instituições financeiras. O DeFi utiliza contratos inteligentes, que são programas autoexecutáveis que facilitam, verificam e aplicam os termos de um contrato.

Aqui estão alguns pontos-chave sobre DeFi:

1. **Descentralização**: Ao contrário das finanças tradicionais, que dependem de instituições centralizadas, o DeFi opera em redes descentralizadas, como Ethereum. Isso significa que não há uma única entidade controlando as operações, o que pode aumentar a segurança e a transparência.

2. **Contratos Inteligentes**: Os contratos inteligentes são fundamentais para o funcionamento do DeFi. Eles automatizam processos financeiros, como empréstimos, trocas de ativos e liquidações, sem a necessidade de intervenção humana.

3. **A